## Imports

In [102]:
import tensorflow as tf
import pandas as pd
import numpy as np
import os

In [92]:
def load_data(path,batch_size,res=128):
  return tf.keras.utils.image_dataset_from_directory(
    path,
    color_mode="rgb",
    batch_size=batch_size,
    image_size=(res,res),
    shuffle=True)

In [ ]:
test_images = load_data("test_images_cropped/",batch_size=128)

# Predicciones Clases

### Cargar modelo entrenado

In [95]:
NUM_CLASSES = 46
model_entrenado = tf.keras.models.load_model('Modelos/mode64_15epocas.h5')

### Predecir clases (1-2 Horas)

In [96]:
diccionario_clases = {}
label_list = []

for class_name in os.listdir('reduced_train_images'):
    diccionario_clases[class_name] = []
    label_list.append(class_name)

In [98]:
RES_ESPACIAL = 64

print("Processing image "+str(1))
test_image = tf.keras.preprocessing.image.load_img("test_images_cropped/test_item0.jpg", target_size=(RES_ESPACIAL,RES_ESPACIAL))
test_array = tf.keras.preprocessing.image.img_to_array(test_image)
test_array = tf.expand_dims(test_array, 0) # Create batch axis
predictions = model_entrenado.predict(test_array, verbose=0)
for i in range(1, int(len(os.listdir('test_images')))):
    print("Processing image "+str(i+1))
    test_image = tf.keras.preprocessing.image.load_img("test_images_cropped/test_item"+str(i)+".jpg", target_size=(RES_ESPACIAL,RES_ESPACIAL))
    test_array = tf.keras.preprocessing.image.img_to_array(test_image)
    test_array = tf.expand_dims(test_array, 0) # Create batch axis
    predictions = np.vstack([predictions, model_entrenado.predict(test_array, verbose=0)])

Processing image 1
Processing image 2
Processing image 3
Processing image 4
Processing image 5
Processing image 6
Processing image 7
Processing image 8
Processing image 9
Processing image 10
Processing image 11
Processing image 12
Processing image 13
Processing image 14
Processing image 15
Processing image 16
Processing image 17
Processing image 18
Processing image 19
Processing image 20
Processing image 21
Processing image 22
Processing image 23
Processing image 24
Processing image 25
Processing image 26
Processing image 27
Processing image 28
Processing image 29
Processing image 30
Processing image 31
Processing image 32
Processing image 33
Processing image 34
Processing image 35
Processing image 36
Processing image 37
Processing image 38
Processing image 39
Processing image 40
Processing image 41
Processing image 42
Processing image 43
Processing image 44
Processing image 45
Processing image 46
Processing image 47
Processing image 48
Processing image 49
Processing image 50
Processin

In [99]:
for i in range(len(predictions)):
    numero_clase_predicha = np.argmax(predictions[i])
    #print("Image "+str(i)+" predicted as "+str(numero_clase_predicha))
    diccionario_clases[label_list[numero_clase_predicha]].append(i)

### Predecir todos anorak

In [ ]:
for i in range(54430):
    numero_clase_predicha = 0
    diccionario_clases[label_list[numero_clase_predicha]].append(i)

### Guardar predicciones

In [100]:
f = open("Predicciones/pred_class.csv", "w")
for key in diccionario_clases.keys():
    f.write(str(key) + ",")
    for i in range(len(diccionario_clases[key])):
        if i == 0:
            f.write(str(diccionario_clases[key][i]))
        else:
            f.write(" " + str(diccionario_clases[key][i]))
    f.write("\n")
f.close()

### Usar predicciones guardadas

In [109]:
diccionario_clases = {}
with open("Predicciones/pred_class.csv") as file:
    for line in file:
        line = line.strip()
        separado = line.split(",")
        class_name = separado[0]
        imagenes = separado[1].split(" ")
        if imagenes[0] == "":
            diccionario_clases[class_name] = []
        else:
            diccionario_clases[class_name] = list(map(int, imagenes))

In [ ]:
for i in diccionario_clases.keys():
    print(i, len(diccionario_clases[i]))

# Predicciones atributos

### Cargar modelo entrenado

In [ ]:
model_entrenado = tf.keras.models.load_model('MODELO_ATRIBUTOS_1EPOCH_10000SAMPLESIZE.h5')
model_entrenado.trainable = False

hidden = tf.keras.layers.Dense(64, activation='relu')(model_entrenado.output)
output = tf.keras.layers.Dense(NUM_CLASSES, activation='softmax')(hidden)
model = tf.keras.Model(inputs=model_entrenado.input, outputs=output)
#model.summary()

### Predecir atributos

In [ ]:
diccionario_atributos = {}

for j in range(1000):
    test_image = tf.keras.preprocessing.image.load_img("test_images/test_item" + str(j) + ".jpg", target_size=(RES_ESPACIAL,RES_ESPACIAL))
    test_array = tf.keras.preprocessing.image.img_to_array(test_image)
    test_array = tf.expand_dims(test_array, 0) # Create batch axis
predictions = model.predict(test_array, verbose=0)

print(predictions)

for i in predictions[0]:
    if i > 0.5:
        print(i)
        print("Atributo encontrado")
        print("Nombre del atributo: ", diccionario_atributos[np.argmax(predictions[0])])
        print("Valor del atributo: ", i)
        print("--------------------------------------------------")

### Predecir todos 0

In [69]:
diccionario_atributos = {}

for i in range(54430):
    diccionario_atributos[i] = [0]*1000

### Guardar predicciones

In [ ]:
f = open("Predicciones/pred_atributes.csv", "w")
for key in diccionario_atributos.keys():
    f.write(str(key) + ",")
    for i in range(len(diccionario_atributos[key])):
        if i == 0:
            f.write(str(diccionario_atributos[key][i]))
        else:
            f.write(" " + str(diccionario_atributos[key][i]))
    f.write("\n")
f.close()

### Usar predicciones guardadas

In [70]:
diccionario_atributos = {}
with open("Predicciones/pred_atributes.csv") as file:
    for line in file:
        line = line.strip()
        separado = line.split(",")
        image_number = separado[0]
        atributos = separado[1].split(" ")
        if imagenes[0] == "":
            diccionario_atributos[image_number] = []
        else:
            diccionario_atributos[image_number] = atributos

## Extraccion colores

In [ ]:
from colorthief import ColorThief

### Generar diccionario colores

In [ ]:
colores = {}

for i in range(54430):
    if i%1000 == 0:
        print(i)
    color_thief = ColorThief("test_images_cropped/test_item" + str(i) + ".jpg")
    colores[i] = np.array(color_thief.get_color(quality=1)) #añade color dominante

## Usar diccionario guardado

diccionario_colores = {}
with open("Predicciones/colores.csv") as file:
    for line in file:
        line = line.strip()
        separado = line.split(",")
        image_number = separado[0]
        atributos = separado[1].split(" ")
        if imagenes[0] == "":
            diccionario_atributos[image_number] = []
        else:
            diccionario_atributos[image_number] = atributos

## Generar archivo con similitudes de cada imagen

In [117]:
def cienSimilaresRandom(id_busqueda,ids_clase):
    lista_similitud = []
    for id_imagen in ids_clase:
        if id_imagen != id_busqueda:
            lista_similitud.append(id_imagen)
    if len(lista_similitud) > 100:
        np.random.shuffle(lista_similitud)
        lista_similitud = lista_similitud[:100]
    else:
        while len(lista_similitud) < 100:
            randomID = np.random.randint(0,54430)
            if randomID not in lista_similitud:
                lista_similitud.append(randomID)
    return lista_similitud


def imagenesSimilares(clases):

    archivo = open("Predicciones/simil_sin_color.csv","w")
    archivo.write("Id,Expected\n")

    for clase in clases.keys():
        ids_clase = clases[clase]
        for id_busqueda in ids_clase:
            if id_busqueda != 8782 and id_busqueda != 45571 and id_busqueda != 54157:
                similitudes = cienSimilaresRandom(id_busqueda,ids_clase) #FUNCION QUE DEVUELVE 100 IMAGENES MAS SIMILARES
                similitudesSTR = " ".join(str(x) for x in similitudes)
                archivo.write(str(id_busqueda) + "," + similitudesSTR + "\n")
    
    archivo.close()

imagenesSimilares(diccionario_clases)

In [105]:
#clases = {"Anorak": [120,40,1], "Blouse": [7,9,5412,1234]}
def generarArchivo(clases, atributos, nombre_archivo="similitudes"):
    
    archivo = open("Predicciones/"+nombre_archivo+".csv", "w")
    archivo.write("Id,Expected\n")

    for clase in clases.keys():
        imagenes_en_clase = clases[clase]
        for id in imagenes_en_clase:
            if id != "8782" and id != "45571" and id != "54157":
                similitudes = imagenes_en_clase.copy()
                # no se parece a si mismo
                similitudes.remove(id)
                # solo las primeras 100
                similitudes = similitudes[:100]
                similitudes = " ".join(str(x) for x in similitudes)
                archivo.write(str(id)+","+str(similitudes)+"\n")

    archivo.close()

generarArchivo(diccionario_clases, diccionario_atributos, "simil_sin_color")

## Mejorar archivo con solucion conocida

#### Cargar datos

In [103]:
similitudes = pd.read_csv('Predicciones/similitudes.csv')
solution = pd.read_csv('sample_solution_key_KAGGLE.csv')

In [18]:
similitudes = pd.read_csv('ejemplo_submission_file.csv')

### Generar archivo final

In [104]:
for i in range(len(similitudes)):
    if similitudes['Id'][i] in solution['Id'].values:
        row = similitudes.iloc[i].values
        #print(type(row[0]))
        #print(row[1])
        #print()
        #print(type(solution[solution['Id'] == similitudes['Id'][i]].values[0][1]))
        #add = row[0] + solution[solution['Id'] == similitudes['Id'][i]].values[0][1] #+ [row[1]]
        similitudes.iloc[i] = solution[solution['Id'] == similitudes['Id'][i]].values[0]
        
similitudes.to_csv('Predicciones/Kaggle_submit.csv', index=False)